# Neural Approach

In [1]:
from nltk.tokenize import sent_tokenize
import string
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import random
import re
import math
import nltk

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [0]:
fp = open("/content/drive/My Drive/Theory_of_Computation/speeches.txt", 'r', encoding="utf-8-sig", errors='ignore')
data = fp.read()

data = data.replace("\n", "")
data = data.replace("...", ". ")
data = re.sub(r"SPEECH [0-9]","", data)
data = re.sub(r"[:;]",".",data)

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
sent_tokenize_list = sent_tokenize(data)

In [0]:
lines = list()
for i in range(len(sent_tokenize_list)):
  if(len(sent_tokenize_list[i])>1):
    lines.append("<s> " + sent_tokenize_list[i][:-1] + " </s>")

In [0]:
random.shuffle(lines)
cut = int(0.8*len(lines))
train = np.array(lines[:cut])
test = np.array(lines[cut:])

In [11]:
train

array(['<s> Six people were killed trying to get him back </s>',
       '<s> WE HAVE A GROUP OF PEOPLE THAT WANT TO GET ELECTED WHO HAVE A RIGGED SYSTEM AND WHO HAVE REALLY RIGGED THE SYSTEM TO A LARGE EXTENT </s>',
       '<s> THESE ARE AMATEURS </s>', ..., '<s> He’s at two </s>',
       '<s> It’s just make America great again </s>',
       '<s> Thousands and thousands of people standing outside of an airplane hangar because the hangar was packed </s>'],
      dtype='<U1968')

In [0]:
def clean_doc(doc):
  table = str.maketrans('', '', string.punctuation)
  text = doc.translate(table)
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

In [0]:
sequences = list()
seq_length = 2

for i in range(len(train)):
#   tokens = ["<start>"]
  tokens = (clean_doc(train[i]))
#   tokens.append("<end>")
  if(len(tokens)>seq_length):
    for k in range(0,len(tokens)-seq_length):
      sequences.append(tokens[k:k+seq_length+1])
#   titles[i] = ' '.join(clean_doc(titles[i]))

In [14]:
sequences[:10]

[['s', 'six', 'people'],
 ['six', 'people', 'were'],
 ['people', 'were', 'killed'],
 ['were', 'killed', 'trying'],
 ['killed', 'trying', 'to'],
 ['trying', 'to', 'get'],
 ['to', 'get', 'him'],
 ['get', 'him', 'back'],
 ['him', 'back', 's'],
 ['s', 'we', 'have']]

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences_intEncoded = tokenizer.texts_to_sequences(sequences)

In [16]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7594


In [17]:
sequences_intEncoded[:5]

[[1, 510, 21], [510, 21, 68], [21, 68, 400], [68, 400, 411], [400, 411, 3]]

In [18]:
sequences_intEncoded = np.array(sequences_intEncoded)
np.shape(sequences)

(136745, 3)

In [0]:
X = sequences_intEncoded[:,:-1]
y = sequences_intEncoded[:,-1]

In [20]:
X[:5]

array([[  1, 510],
       [510,  21],
       [ 21,  68],
       [ 68, 400],
       [400, 411]])

In [0]:
y = to_categorical(y, num_classes=vocab_size)

In [22]:
###### VANILLA RECURRENT NEURAL NETWORK MODEL ######
from keras.layers import SimpleRNN

model1 = Sequential()
model1.add(Embedding(vocab_size, 50, input_length=seq_length))
model1.add(SimpleRNN(25, return_sequences=True))
model1.add(SimpleRNN(25))
model1.add(Dense(50, activation='relu'))
model1.add(Dense(vocab_size, activation='softmax'))

model1.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 50)             379700    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 2, 25)             1900      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 25)                1275      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1300      
_________________________________________________________________
dense_2 (Dense)              (None, 7594)              387294    
Total params: 771,469
Trainable params: 771,469
Non-trainable params: 0
_________________________________________________________________


In [23]:
###### LONG SHORT TERM MEMORY NEURAL NETWORK MODEL ######
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(25, return_sequences=True))
model.add(LSTM(25))
model.add(Dense(50, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2, 50)             379700    
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 25)             7600      
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                1300      
_________________________________________________________________
dense_4 (Dense)              (None, 7594)              387294    
Total params: 780,994
Trainable params: 780,994
Non-trainable params: 0
_________________________________________________________________


In [24]:
###### TRAINING LONG SHORT TERM MEMORY NEURAL NETWORK MODEL ######

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=40)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/40
136745/136745 [==============================] - 64s 469us/step - loss: 6.2121 - acc: 0.0913
Epoch 2/40
136745/136745 [==============================] - 62s 454us/step - loss: 5.7634 - acc: 0.1022
Epoch 3/40
136745/136745 [==============================] - 62s 452us/step - loss: 5.3309 - acc: 0.1487
Epoch 4/40
  4352/136745 [..............................] - ETA: 59s - loss: 5.1072 - acc: 0.1691

KeyboardInterrupt: ignored

In [27]:
###### TRAINING VANILLA RECURRENT NEURAL NETWORK MODEL ######

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(X, y, batch_size=128, epochs=40)

Epoch 1/40
136745/136745 [==============================] - 57s 419us/step - loss: 5.7670 - acc: 0.1245
Epoch 2/40
136745/136745 [==============================] - 56s 412us/step - loss: 4.9913 - acc: 0.1901
Epoch 3/40
136745/136745 [==============================] - 57s 418us/step - loss: 4.7305 - acc: 0.2060
Epoch 4/40
136745/136745 [==============================] - 55s 402us/step - loss: 4.5378 - acc: 0.2192
Epoch 5/40
 21376/136745 [===>..........................] - ETA: 45s - loss: 4.3710 - acc: 0.2323

KeyboardInterrupt: ignored

**Random text of 5 sentences**

In [30]:
###### LSTM OUTPUT WORDS ######

total_score = 0
total_count = 0
generated_test_lstm = []
count = 0

for i in range(len(test)):
  result = list()
  text = clean_doc(test[i])
  
  if(len(text)>2):
    result = text[:seq_length]
    r = -1*seq_length
    for _ in range(2,len(text)):
      encoded = tokenizer.texts_to_sequences([' '.join(result[r:])])[0]
      if(len(encoded)<seq_length):
        for j in range(len(encoded),seq_length):
          encoded.append(0)
      encoded = np.array([encoded])
      yhat = model.predict_classes(encoded, verbose=0)
      out_word = ''
      for word, index in tokenizer.word_index.items():
        if index == yhat:
          out_word = word
          break
      result.append(out_word)
    
    generated_test_lstm.append(result)
    #print(test[i])

    if(count < 5): ### to print 5 sentences only
      print(result)
    count+=1

['s', 'but', 'i', 'have', 'a', 'lot', 'of']
['s', 'no', 's', 'a', 'lot', 'of', 'the', 'lot', 'of', 'the', 'lot', 'of', 'the']
['s', 'youve', 'have', 'a', 'lot', 'of', 'the']
['s', 'its', 'the', 'lot', 'of']
['s', 'but', 'i', 'have', 'a', 'lot', 'of', 'the', 'lot', 'of', 'the', 'lot', 'of', 'the', 'lot', 'of', 'the', 'lot', 'of', 'the', 'lot', 'of', 'the', 'lot']


In [31]:
##### VANILLA RNN OUTPUT WORDS #######

total_score = 0
total_count = 0
generated_test_rnn = []
count = 0

for i in range(len(test)):
  result = list()
  text = clean_doc(test[i])
  
  if(len(text)>2):
    result = text[:seq_length]
    r = -1*seq_length
    for _ in range(2,len(text)):
      encoded = tokenizer.texts_to_sequences([' '.join(result[r:])])[0]
      if(len(encoded)<seq_length):
        for j in range(len(encoded),seq_length):
          encoded.append(0)
      encoded = np.array([encoded])
      yhat = model1.predict_classes(encoded, verbose=0)
      out_word = ''
      for word, index in tokenizer.word_index.items():
        if index == yhat:
          out_word = word
          break
      result.append(out_word)
    
    generated_test_rnn.append(result)
    #print(test[i])

    if(count < 5): ### to print 5 sentences only
      print(result)
    count+=1

['s', 'but', 'i', 'm', 'going', 'to', 'be']
['s', 'no', 'i', 'm', 'going', 'to', 'be', 'a', 'lot', 'of', 'the', 'people', 's']
['s', 'youve', 'been', 'going', 'to', 'be', 'a']
['s', 'its', 'going', 'to', 'be']
['s', 'but', 'i', 'm', 'going', 'to', 'be', 'a', 'lot', 'of', 'the', 'people', 's', 'going', 'to', 'be', 'a', 'lot', 'of', 'the', 'people', 's', 'going', 'to']


**Calculation of Perplexity**

In [0]:
file1 = open("/content/drive/My Drive/Theory_of_Computation/speeches.txt", 'r', encoding="utf-8-sig", errors='ignore')
train_1 = file1.read()
train_1 = train_1.lower()

In [35]:
t1 = sent_tokenize(train_1)
for i in range (len(t1)):
  t1[i] = re.sub(r'[^A-Za-z\s\']+', "", t1[i])
  t1[i] = '<s> '+t1[i]+' </s>'
train = t1[:13000] # 80% of dataset is train
test = t1[13000:] # 20% of dataset is test
print(len(t1))

16401


In [0]:
from collections import Counter
f_1 = {} #bigram
for i in range (len(train)):
  tokens = train[i].split()
  bigrams = nltk.bigrams(tokens)
  f = dict(nltk.FreqDist(bigrams))
  f_1 = dict(Counter(f)+Counter(f_1))

In [0]:
train_1 = re.sub(r'[^A-Za-z\s\']+', "", train_1)
ls = train_1.split()
d = {}
for i in range (len(ls)):
  if ls[i] not in d:
    d[ls[i]] = 1
  else:
    d[ls[i]] += 1
Vocab = len(d)
Token = sum(d.values())

In [0]:
def bigram(w1,w2):
  if (w1+" "+w2) not in f_1:
    if w1 in d.keys():
      return 1/(d[w1]+Vocab) # Add-1 Smoothing
    else:
      return 1/(Vocab) # Add-1 Smoothing
  else:
    if w1 in d.keys():
      return (f_1(w1+" "+w2)+1)/(d[w1]+Vocab) # Add-1 Smoothing
    else:
      return (f_1(w1+" "+w2)+1)/(Vocab) # Add-1 Smoothing

In [0]:
def bigram_sentence_probability(sentence):
  bigram_sentence_probability_log_sum = 0
  previous_word = None
  for word in sentence:
    if previous_word!=None:
      x = bigram(previous_word,word)
      bigram_sentence_probability_log_sum += math.log(x,2)
    previous_word = word
  return math.pow(2, bigram_sentence_probability_log_sum)

In [0]:
def calculate_number_of_bigrams(sentences):
  bigram_count = 0
  for sentence in sentences:
    bigram_count += len(sentence) - 1
  return bigram_count

In [0]:
def calc_bigram_perplexity(sentences):
  bigram_count = calculate_number_of_bigrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(bigram_sentence_probability(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / bigram_count)

In [49]:
print("Perplexity of test corpus with respect to bigram model is:",end = " ")
print(calc_bigram_perplexity(test))

print("Perplexity of test corpus generated by LSTM architecture with respect to bigram model is:",end = " ")
print(calc_bigram_perplexity(generated_test_lstm))

Perplexity of test corpus with respect to bigram model is: 147.40810983034604
Perplexity of test corpus generated by LSTM architecture with respect to bigram model is: 6096.576391520106


In [51]:
print("Perplexity of test corpus with respect to bigram model is:",end = " ")
print(calc_bigram_perplexity(test))

print("Perplexity of test corpus generated by Baseline RNN architecture with respect to bigram model is:",end = " ")
print(calc_bigram_perplexity(generated_test_rnn))

Perplexity of test corpus with respect to bigram model is: 147.40810983034604
Perplexity of test corpus generated by Baseline RNN architecture with respect to bigram model is: 5977.603062968707


# Classical Approach

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.util import ngrams
from collections import Counter
import re
import numpy as np
import random
import math
import operator
import string

In [0]:
file2 = open("/content/drive/My Drive/Theory_of_Computation/speeches.txt", 'r', encoding="utf-8-sig", errors='ignore')
train2 = file2.read()
train2 = train2.lower()

In [54]:
t1 = sent_tokenize(train2)
for i in range (len(t1)):
  t1[i] = re.sub(r'[^A-Za-z\s\']+', "", t1[i])
  t1[i] = '<s> '+t1[i]+' </s>'
train = t1[:13000] 
test = t1[13000:]
print(len(t1))

16401


In [0]:
f_1 = {} 
for i in range (len(train)):
  tokens = train[i].split()
  bigrams = nltk.bigrams(tokens)
  f = dict(nltk.FreqDist(bigrams))
  f_1 = dict(Counter(f)+Counter(f_1))

In [0]:
f_2 = {}
for i in range (len(train)):
  tokens = train[i].split()
  trigrams = nltk.trigrams(tokens)
  f = dict(nltk.FreqDist(trigrams))
  f_2 = dict(Counter(f)+Counter(f_2))

In [0]:
from nltk import everygrams
f_3 = {}
for i in range (len(train)):
  tokens = train[i].split()
  fourgrams = list(everygrams(tokens,4,4)) 
  f = dict(nltk.FreqDist(fourgrams))
  f_3 = dict(Counter(f)+Counter(f_3))

In [0]:
train2 = re.sub(r'[^A-Za-z\s\']+', "", train2)
ls = train2.split()
d = {}
for i in range (len(ls)):
  if ls[i] not in d:
    d[ls[i]] = 1
  else:
    d[ls[i]] += 1
Vocab = len(d)
Token = sum(d.values())

In [0]:
def unigram(w1):
  if w1 not in d:
    return 1/(Vocab+Token) # Add-1 Smoothing
  else:
    return (d[w1]+1)/(Vocab+Token) # Add-1 Smoothing

In [0]:
def bigram(w1,w2):
  if (w1+" "+w2) not in f_1:
    if w1 in d.keys():
      return 1/(d[w1]+Vocab) # Add-1 Smoothing
    else:
      return 1/(Vocab) # Add-1 Smoothing
  else:
    if w1 in d.keys():
      return (f_1(w1+" "+w2)+1)/(d[w1]+Vocab) # Add-1 Smoothing
    else:
      return (f_1(w1+" "+w2)+1)/(Vocab) # Add-1 Smoothing

In [0]:
def trigram(w1,w2,w3):
  if (w1+" "+w2+" "+w3) not in f_2:
    if w1+" "+w2 in f_1.keys():
      return 1/(f_1[w1+" "+w2]+Vocab) # Add-1 Smoothing
    else:
      return 1/(Vocab) # Add-1 Smoothing
  else:
    if w1+" "+w2 in f_1.keys():
      return (f_2(w1+" "+w2+" "+w3)+1)/(fdist1[w1+" "+w2]+Vocab) # Add-1 Smoothing
    else:
      return (f_2(w1+" "+w2+" "+w3)+1)/(Vocab) # Add-1 Smoothing

In [0]:
def quadgram(w1,w2,w3,w4):
  if (w1+" "+w2+" "+w3+" "+w4) not in f_3:
    if w1+" "+w2 + " "+w3 in f_2.keys():
      return 1/(f_2[w1+" "+w2+" "+w3]+Vocab) # Add-1 Smoothing
    else:
      return 1/(Vocab) # Add-1 Smoothing
  else:
    if w1+" "+w2 + " "+w3 in f_2.keys():
      return (f_3(w1+" "+w2+" "+w3+" "+w4)+1)/(f_2[w1+" "+w2+" "+w3]+Vocab) # Add-1 Smoothing
    else:
      return (f_3(w1+" "+w2+" "+w3+" "+w4)+1)/(Vocab) # Add-1 Smoothing

In [0]:
def unigram_sentence_probability(sentence):
  sentence_probability_log_sum = 0
  for word in sentence:
    x = unigram(word)
    sentence_probability_log_sum += math.log(x,2)
  return math.pow(2, sentence_probability_log_sum)

In [0]:
def bigram_sentence_probability(sentence):
  bigram_sentence_probability_log_sum = 0
  previous_word = None
  for word in sentence:
    if previous_word!=None:
      x = bigram(previous_word,word)
      bigram_sentence_probability_log_sum += math.log(x,2)
    previous_word = word
  return math.pow(2, bigram_sentence_probability_log_sum)

In [0]:
def trigram_sentence_probability(sentence):
  trigram_sentence_probability_log_sum = 0
  previous_word = None
  previous_previous_word = None
  for word in sentence:
    if previous_word!=None and previous_previous_word!=None:
      x = trigram(previous_previous_word,previous_word,word)
      trigram_sentence_probability_log_sum += math.log(x,2)
    previous_previous_word = previous_word
    previous_word = word
  return math.pow(2, trigram_sentence_probability_log_sum)

In [0]:
def quadgram_sentence_probability(sentence):
  quadgram_sentence_probability_log_sum = 0
  previous_word = None
  previous_previous_word = None
  previous_previous_previous_word = None
  for word in sentence:
    if previous_word!=None and previous_previous_word!=None and previous_previous_previous_word!=None :
      x = quadgram(previous_previous_previous_previous_previous_word,previous_word,word)
      quadgram_sentence_probability_log_sum += math.log(x,2)
    previous_previous_previous_word = previous_previous_word
    previous_previous_word = previous_word
    previous_word = word
  return math.pow(2, quadgram_sentence_probability_log_sum)

In [0]:
def calculate_number_of_unigrams(sentences):
  unigram_count = 0
  for sentence in sentences:
    # remove two for <s> and </s>
    unigram_count += len(sentence) - 2
  return unigram_count
def calculate_number_of_bigrams(sentences):
  bigram_count = 0
  for sentence in sentences:
    bigram_count += len(sentence) - 1
  return bigram_count
def calculate_number_of_trigrams(sentences):
  trigram_count = 0
  for sentence in sentences:
    trigram_count += len(sentence) - 2
  return trigram_count
def calculate_number_of_quadgrams(sentences):
  quadgram_count = 0
  for sentence in sentences:
    quadgram_count += len(sentence) - 3
  return quadgram_count

In [0]:
def calc_unigram_perplexity(sentences):
  unigram_count = calculate_number_of_unigrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(unigram_sentence_probability(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / unigram_count)

In [0]:
def calc_bigram_perplexity(sentences):
  bigram_count = calculate_number_of_bigrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(bigram_sentence_probability(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / bigram_count)

In [0]:
def calc_trigram_perplexity(sentences):
  trigram_count = calculate_number_of_bigrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(trigram_sentence_probability(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / trigram_count)

In [0]:
def calc_quadgram_perplexity(sentences):
  quadgram_count = calculate_number_of_quadgrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(quadgram_sentence_probability(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / quadgram_count)

**Calculation of Perplexity**

In [108]:
import math
print("Perplexity of test corpus with respect to unigram model is:",end = " ")
print(calc_unigram_perplexity(test))
print("Perplexity of test corpus with respect to bigram model is:",end = " ")
print(calc_bigram_perplexity(test))
print("Perplexity of test corpus with respect to trigram model is:",end = " ")
print(calc_trigram_perplexity(test))
print("Perplexity of test corpus with respect to quadgram model is:",end = " ")
print(calc_quadgram_perplexity(test))


Perplexity of test corpus with respect to unigram model is: 186.64101133266124
Perplexity of test corpus with respect to bigram model is: 147.40810983034604
Perplexity of test corpus with respect to trigram model is: 135.92047675682477
Perplexity of test corpus with respect to quadgram model is: 1.0


In [0]:
#Classical generator to generate new words using a n_gram model
def mle_generator(n_gram, initial_sequence):
  sentence = ["<s>"]
  if(n_gram==1):
    for i in range(20):
      max_prob = 0
      max_prob_list = list()
      for j in d.keys():
        k = (unigram(j))
        if(k>max_prob):
          max_prob = k
          max_prob_list = [j]
        elif(k==max_prob):
          max_prob_list.append(j)
      samples = np.random.multinomial(20,[max_prob]*len(max_prob_list),size=1)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(max_prob_list[index])
  else:
    sentence.extend(initial_sequence)
    i = len(initial_sequence)
    while(sentence[-1]!="</s>" and i<20):
      max_prob = 0
      max_prob_list = list()
      for j in d.keys():
        word_list = sentence[-n_gram+1:]
        word_list.append(j)
        if(n_gram==2):
          k = bigram(word_list[0],word_list[1])
        elif(n_gram==3):
          k = trigram(word_list[0],word_list[1],word_list[2])
        elif(n_gram==4):
          k = quadgram(word_list[0],word_list[1],word_list[2],word_list[3])
        if(k>max_prob):
          max_prob = k
          max_prob_list = [j]
        elif(k==max_prob):
          max_prob_list.append(j)
      samples = np.random.multinomial(20,[max_prob]*len(max_prob_list),size=1)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(max_prob_list[index])
      i+=1
    
  return sentence

In [132]:
print(" ".join(mle_generator(3,["it", "should", "not"])))


<s> it should not speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech speech


**Random text generation of 5 sentences**

In [0]:
# GENERATING RANDOM TEXT:
# UNIGRAMS
print("UNIGRAM: ", end = " ")
print(" ".join(MLE_Generator(1,[])))
# BIGRAM
print("BIGRAM: ", end = " ")
print(" ".join(MLE_Generator(2,["it", "is"])))
# TRIGRAM
print("TRIGRAM: ", end = " ")
print(" ".join(MLE_Generator(3,["there", "is", "something"])))
# QUADGRAM
print("QUADGRAM1: ", end = " ")
print(" ".join(MLE_Generator(4,["but", "there", "is", "no"])))
# ANOTHER QuADGRAM
print("QUADGRAM2: ", end = " ")
print(" ".join(MLE_Generator(4,["it", "is", "to", "be"])))

UNIGRAM:  <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s>
BIGRAM:  <s> it is a lot of the world </s>
TRIGRAM:  <s> there is something that i can tell you that </s>
QUADGRAM1:  <s> but there is no leadership at the top </s>
QUADGRAM2:  <s> it is to be treated a little bit </s>
